<a id=top></a>
# Lesson 0: Working on TIKE, with Cloud Data

## Learning Goals: 
- Learn about TIKE and working on a cloud platform.
- Define cloud terminology: what is a “bucket” or a server? For that matter, what is the “cloud”?
- Access data through astroquery by name, region, or criteria
- Download TESS data and show an image

<!-- ## Lesson Outline:
- Go over TIKE: what can it do? What are its limits (memory, processing power, etc.)?
- Cloud overview. What does it mean to work “on the cloud”? How is this different from working on a laptop? [Provide motivation: why are we doing this?]
- Demo cloud data access using astroquery. Meaningfully interact with data (maybe just read FITS headers) so people see they don’t need to perform a download.
 -->

## What is TIKE?

TIKE stands for the *Timeseries Integrated Knowledge Engine*.

TIKE uses a web-based platform, called JupyterHub, to allow you to run [Jupyter Notebooks](https://jupyterlab.readthedocs.io/en/latest/) and other software "on the cloud" using your web browser: no need to install anything on your local computer. TIKE has access to a cloud copy of the [MAST Archive](https://archive.stsci.edu), enabling researchers (or students!) to access and analyze data from NASA's [TESS mission](https://archive.stsci.edu/missions-and-data/tess). 

TIKE is continually maintained and updated by humans, so if you run into issues please let us know. Don't hesitate to send us your suggestions for packages and tutorials, either through the [MAST help desk](mailto:archive@stsci.edu) or [Project TIKEBook repository](https://github.com/spacetelescope/tike_content).

## What is the "cloud"?

The "cloud" is a term used to describe a global network of servers, each with a unique function. The cloud is not a physical entity, but instead is a vast network of remote servers around the globe which are hooked together and meant to operate as a single data ecosystem. In other words, the cloud refers to servers that are accessed over the internet, and the software and databases that run on those servers. 

In our case, TIKE is a cloud service that runs "in proximity to" MAST data. In practice, this means that the data is not transmitted over the internet, but rather within a data center. This leads to faster access since you don't need to perform a traditional download to move the data to your machine.

### Why would I want to work on the cloud?
Using the cloud has several benefits; principally, as mentioned above, there's no need to download data to your local machine. This saves time, and allows you to perform analyses that wouldn't be possible without a major upgrade to your hard drive capacity. You can access data whenever and wherever you want to, from any device, as long as you have an internet connection.

### What's the difference between working on the cloud and working on TIKE?
Although you might choose to work directly with data stored on the cloud, it can be complex to configure such a system. TIKE handles this complexity, making it as easy as opening a Jupyter Notebook.

## How do I access the cloud?
There are many options for accessing data on the cloud, including command line tools and various Python packages. For this lesson, we will be using a Python data query package called [Astroquery](https://astroquery.readthedocs.io/en/latest/). 

Astroquery is a Python package inside the [Astropy Project](https://www.astropy.org/), which is a community-driven "core package" for doing astronomy with Python. Astroquery is a package within this ecosystem that allows users to access a variety of astronomical data archives. We will use it to access TESS data on the cloud hosted by MAST. 

## Astroquery demonstration

### Imports
The following cell holds the imported packages. These packages are necessary for running the rest of the cells in this notebook, and you can expect to use some of these packages almost everytime you do astronomical research. A description of each import is as follows:

* `numpy` to handle array mathematics
* `pandas` to handle date conversions
* `fits` from astropy.io for accessing FITS files
* `Table` from astropy.table for creating tidy tables of the data
* `WCS` from astropy.wcs for storing World Coordinate Systems information 
* `SkyCoord` from astropy.coordinates for defining RA and Dec for targets
* `matplotlib.pyplot` for plotting data and images
* `Mast` and `Observations` from astroquery.mast for querying data and observations from the MAST archive

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astroquery.mast import Observations


import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

### Querying MAST Data in the Cloud
We will be using the Observations class in the astroquery.mast subpackage from Astroquery. We will show you a few different ways to access data on an object in the TESS data products. 
Accessing data can be done with the following functions:
- `query_object()`
- `query_region()`
- `query_criteria()`

We'll discuss some of their differences and similarities below. But first, we need to enable cloud data access. Fortunately, this is a one-line command.

In [ ]:
Observations.enable_cloud_dataset()

### Choosing a target
First, we have to choose a star from which we'd like to get TESS data. It is best practice to reduce the size of the data you are accessing by selecting only data within a certain radius of the target; by default, this is 0.2 degrees.

Let's start with the star Fomalhaut, the brightest star in the southern constellation of Piscis Austrinus, the "Southern Fish", and one of the brightest stars in the night sky.

<img src="https://upload.wikimedia.org/wikipedia/commons/a/ae/Heic0821f.jpg" width="300">

`Image Credit: NASA, ESA, and the Digitized Sky Survey 2. Acknowledgment: Davide De Martin (ESA/Hubble)`


Fomalhaut is located at Right ascension: 22h 57m 39.0465s, Declination: −29° 37′ 20.050″ with an apparent magnitude in the V-band of 1.16. 

### query_region( )
This function allows us to choose a set of coordinates and a radius, then query all the data that MAST has inside that region. We will use the coordinates for Fomalhaut with a 2 arcsecond radius around it. 

Note: unless you have a specific reason for using `query_region()` (e.g. you already have a list of target coordinates or have a target that doesn't easily resolve), it's often much easier to use a different query function.

In [ ]:
# Entering our coorindates from above
coordinates = SkyCoord("22h57m39.04625s -29d37m20.0533s", frame = "icrs") #defaults to ICRS frame 

# Search on coordinates with 2 second radius
obs = Observations.query_region(coordinates, radius = "2s")

#Let's look at the first five rows
print(obs[:5])

This is just a warmup search, so we don't need to interpret these results too closely just yet.

### query_object()
This type of query is the "simplest", in the sense that we only need to give the name of our target. In doing so, we're trusting `query_object` to unambiguously resolve our target to coordinates on the sky. 

**Note:** In a vanishingly small but non-zero number of cases, a target may be incorrectly resolved. Double-check the target name before defending any PhD dissertations.

In [ ]:
# Query MAST for TESS observations of Fomalhaut 
obs = Observations.query_object("Fomalhaut", radius="1s")

# Print out the observations we have queried, with limited columns
limit_cols = ['obs_collection', 'target_name','distance', 't_exptime']
obs[limit_cols][:5]

We limit the columns in the `obs` output above for simplicity. Feel free to print the full table if you're curious.

Let's try limiting the results to those from the TESS mission by filtering on our table.

In [ ]:
#Let's parse these observations for just those from TESS
tess_obs = (obs['obs_collection'] == "TESS")

# Print out the observations we have queried 
obs[tess_obs][limit_cols]

This is nice that we can filter on Observations after getting a table of them, but we can use a different function to get there faster. Before we get to that, let's do a quick exercise to test your knowledge.

#### Exercise 1
Using the `query_object()` method, how many MAST Observations are within 2 arcseconds of Trappist-1?

### query_criteria( )
We can give `query_criteria()` a larger number of parameters to refine the Observations in just one line of code. Feel free to explore the [list of these queryable parameters](https://mast.stsci.edu/api/v0/_productsfields.html), but we'll discuss these in greater detail later.

In [ ]:
TESS_table = Observations.query_criteria(objectname="Fomalhaut",radius="2 arcsec", obs_collection="TESS")

# Let's print out some relevant columns of this table
TESS_table[limit_cols]

#### Exercise 2
How could you tell if the query returned no Observations?

### Plot a Full Frame Image
We can see from the `target_name` column that some of the data TESS has on Fomalhaut are Full Frame Images (more on these later). These will be stored in FITS files which can be downloaded and viewed on TIKE. 

#### What is a FITS file?
File Image Transport System (FITS) is a file format designed to store, transmit, and manipulate scientific images and associated data. It is the most widely used file type in astronomical research. A FITS file consists of one or more Header + Data Units (HDUs), where the first HDU is called the primary HDU, or primary array. The primary array may be empty or contain an N-dimensional array of pixels, such as a 1-D spectrum, a 2-D image, or a 3-D data cube. 

For more info on FITS files, see the [NASA FITS Support Office](https://fits.gsfc.nasa.gov/). 

In [ ]:
# Query the observations from MAST to get a list of products for our selected observations
data_products = Observations.get_product_list(TESS_table)

# Get only the image type products
filtered = Observations.filter_products(data_products, dataproduct_type ='image')

# Look at the first five results
filtered[:5]

There are thousands of TESS images available. Let's select a calibrated full frame image; for reproducability, we'll pick the product corresponding to `obs_id=tess2018235152941-s0002-1-1-0121-s`.

In [ ]:
fixed_id = "tess2018235152941-s0002-1-1-0121-s"

filtered_image = Observations.filter_products(data_products, obs_id = fixed_id, description='Calibrated full frame image')

# Confirm we have the product we want
filtered_image

Now we can proceed to the download.

In [ ]:
data = Observations.download_products(filtered_image)

# Save the local path for later
fits_file = data['Local Path'][0]

### Understanding the FITS FFI structure

TESS FFI FITS files contain a primary HDU with metadata stored in the header.  The first extension HDU contains more metadata in the header, and stores the full frame image.  The second extension HDU contains the uncertainty values for the image.  Let's examine the structure of the FITS file using the astropy.fits `info` function, which shows the FITS file format in more detail.

In [ ]:
fits.info(fits_file)

### Reading Data from the FITS File

HDU 1 contains the calibrated image data that we'll need to plot an image, alongside information about the data and how it was collected.

Now that we have the file, let's store the world coordinate system information for use later. World Coordinate Systems (WCSs) describe the geometric transformations between one set of coordinates and another, almost all the FITS files you come across will have a WCS in its metadata. We can use the astropy.wcs WCS function to store the information from the image extension HDU's header. 

In [ ]:
# Open the file, extract the WCS and Image data, and then close the file
with fits.open(fits_file, mode = "readonly") as hdulist:
    image = hdulist[1].data     # Get the actual image data
    header = hdulist[1].header  # Get the additional information in the header
    wcs = WCS(header)           # Extract the WCS information from the header
    
# Preview the first ten items in the header
header[:10]

### Display the Image

Finally, let's plot our full frame image of Fomalhaut. 

In [ ]:
# Create a 12x12 figure
plt.figure(figsize = (12,12))

# Use our WCS information to set the coordinates
plt.subplot(projection = wcs)

# Plot the image, adjusting some settings along the way
plt.imshow(image
           , vmin = np.percentile(image,4)    # Set the minimum brightness to the 4th percentile of the image
           , vmax = np.percentile(image, 98)  # Set the max to the 98th percentile
           , cmap = 'inferno'                 # Pick the color scheme
          )

# Create some labels for our axes
plt.xlabel('RA')
plt.ylabel('Dec')

Done! Fomalhaut is fairly obvious as the brightest star in this image, with large "spikes" around the image. The glow in the lower left of the image is caused by stray light (likely from the Earth or moon) on the TESS cameras.

## Homework
1. Choose a star from this list:
    - Proxima Centauri
    - Procyon (* alf CMi)
    - Aldebaran (* alf Tau)
    - Polaris (alf UMi)
    - Rigel (* bet Ori)
    
Next, query TESS observations for it, download a fits file for 1 calibrated FFI, and then print out the FITS Header. Answer the following questions using the code above as an example. HINT: you may need to find the coordinates for your chosen star, [SIMBAD](https://simbad.u-strasbg.fr/simbad/sim-fid) is a reliable source. 


- What is the date of this observation?
- What are CRVAL1 and CRVAL2?
- How many TESS observations of this star are there?
- What are the exposure times of these observations?

2. Using the data you queried, plot a FITS FFI for the star. 

## Additional Resources
Can't get enough? Here are some links to more information!

If you need an introduction (or a refresher!) to basic Python syntax, there are several great resources available online. [CodeAcademy](https://www.codecademy.com/learn/learn-python-3) is a great service with a totally free option for getting started with Python, note you will have to create an account to use it. Additionally, the Youtube channel FreeCodeCamp.org has a great [video tutorial](https://www.youtube.com/watch?v=rfscVS0vtbw) on everyting you need to get started programming in Python. Another good resource is the [Python 4 Everyone](https://www.py4e.com/) book. 

The full astropy documentation can be found [here](https://docs.astropy.org/en/stable/index.html).

For more info on FITS files, here is a link to the [FITS NASA site](https://fits.gsfc.nasa.gov/). 

SIMBAD is a web-based query service from the University of Strausberg, it is a great resource for getting quick info on stars and other astronomical targets. Here is the link to [Fomalhaut's SIMBAD page](https://simbad.u-strasbg.fr/simbad/sim-id?Ident=fomalhaut&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id)


## What's next?

The next lesson will use what we have learned about gathering TESS data to plot a light curve of an exoplanet. We'll also explore time series data and their applications. This will involve learning how exoplanets are detected and discover what other types of systems can be analyzed using light curves. Stay tuned!


## Acknowldegements

If you write a paper using TESS data from MAST, please acknowledge this using the following template:

> This paper includes data collected with the TESS mission, obtained from the MAST data archive at the Space Telescope Science Institute (STScI). Funding for the TESS mission is provided by the NASA Explorer Program. STScI is operated by the Association of Universities for Research in Astronomy, Inc., under NASA contract NAS 5–26555.

Any published work that uses Astroquery should include a citation which can be found at [this link](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION), or can be printed out in a code cell with: `astroquery.__citation__` as long as the astroquery package is imported. 

### About this Notebook:
If you have comments or questions on this notebook, please contact us through the [Archive Help Desk e-mail](mailto:archive@stsci.edu).

**Author:** Emma Lieb

**Last Updated:** June 2023

[Top of Page](#top)

<img style=float:right; src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 